# Car Price Predictor Machine Learning Project 

### Step 1: Web Scraping Data

Website: Cars24 

link: https://www.cars24.com/buy-used-car?sort=bestmatch&serveWarrantyCount=true&storeCityId=2&pinId=110001

Main Libraries Used: Selenium, BeautifulSoup

In [1]:
import selenium
import time
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless") 
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
url="https://www.cars24.com/buy-used-car?sort=bestmatch&serveWarrantyCount=true&storeCityId=2&pinId=110001"

In [3]:
def get_details(soup):
    n=soup.find_all('h2',{"class":"_2lmIw"})
    g=soup.find_all('ul',{"class":"_1hOnS"})
    h=soup.find_all('ul',{"class":"_13yb6"})
    p=soup.find_all('div',{"class":"_18ToE"})
    
    year=[]
    brand=[]
    name=[]
    transmission=[]
    distance=[]
    ownership=[]
    fuel=[]
    price=[]
    
    for i in n:
        i=i.text
        year.append(int(i[:4]))
        brand.append(i.split(" ")[1])
        name.append(i[5:])
        
    for ul in g:
        transmission.append(ul.find_all('li')[-1].text)
        
    for ul in h:
        distance.append(int(ul.find_all('li')[0].text[:-3].replace(',', '')))
        ownership.append(ul.find_all('li')[1].text)
        fuel.append(ul.find_all('li')[2].text)
        
    for div in p:
        price.append(int(div.find_all('span')[0].text[1:].replace(',', '')))
        
    return year,brand,name,transmission,distance,ownership,fuel,price

In [4]:
def Web_Scrapping(url):
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = webdriver.Chrome('C:/Users/91981/Desktop/Python programs/chromedriver')
    driver.get(url)
    time.sleep(2)               # Allow 2 seconds for the web page to open
    scroll_pause_time = 0.5 
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 


    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    
    year,brand,name,transmission,distance,ownership,fuel,price=get_details(soup)
    df=pd.DataFrame({"Year of Manufacture":year,"Brand":brand,"Name":name,"Transmission":transmission,
                   "Distance Travelled(km)":distance,"Ownership":ownership,"Fuel":fuel,"Price(Rs)":price})
    return df

In [5]:
df=Web_Scrapping(url)
df

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 98
Current browser version is 116.0.5845.96 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00D77AC3+2587331]
	Ordinal0 [0x00D0ADD1+2141649]
	Ordinal0 [0x00C03BB8+1063864]
	Ordinal0 [0x00C23638+1193528]
	Ordinal0 [0x00C1F4B5+1176757]
	Ordinal0 [0x00C1CC51+1166417]
	Ordinal0 [0x00C4D12F+1364271]
	Ordinal0 [0x00C4CD5A+1363290]
	Ordinal0 [0x00C484A6+1344678]
	Ordinal0 [0x00C253F6+1201142]
	Ordinal0 [0x00C262E6+1204966]
	GetHandleVerifier [0x00F1DF22+1680738]
	GetHandleVerifier [0x00FD0DBC+2413564]
	GetHandleVerifier [0x00E0D151+563089]
	GetHandleVerifier [0x00E0BF13+558419]
	Ordinal0 [0x00D1081E+2164766]
	Ordinal0 [0x00D15508+2184456]
	Ordinal0 [0x00D15650+2184784]
	Ordinal0 [0x00D1F5BC+2225596]
	BaseThreadInitThunk [0x76417D59+25]
	RtlInitializeExceptionChain [0x77B3B79B+107]
	RtlClearBits [0x77B3B71F+191]


In [ ]:
df.to_csv("Cars24.csv")

### Step 2: Data Analysis

Main Libraries Used: Python, Matplotlib, Seaborn

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd

In [ ]:
df=pd.read_csv("Cars24.csv")
df=df.drop(["Unnamed: 0"],axis=1)
df

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(x="Brand",data=df).set_title('Number of Cars per Brand')

In [ ]:
plt.figure(figsize=(17,8))
sns.jointplot(x="Distance Travelled(km)",y="Price(Rs)",data=df)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Effect of Transmission, Ownership, Fuel on Price of Car')
sns.boxplot(ax=axes[0], data=df, x='Transmission', y='Price(Rs)')
sns.boxplot(ax=axes[1], data=df, x='Ownership', y='Price(Rs)')
sns.boxplot(ax=axes[2], data=df, x='Fuel', y='Price(Rs)')

In [ ]:
plt.figure(figsize=(13,6))
sns.pointplot(x="Year of Manufacture",y="Price(Rs)",data=df).set_title('Year of Manufacture vs Price')

In [ ]:
plt.figure(figsize=(18,6))
sns.boxplot(data=df, x='Brand', y='Price(Rs)').set_title('Brand vs Price')

### Step 3: Data Modelling

Libraries Used: Scikit-Learn

In [ ]:
import warnings
warnings.filterwarnings('ignore')

df["Transmission"]=df["Transmission"].map({"Manual":0,"Automatic":1})
o={"1st Owner":0,"2nd Owner":0.5,"3rd Owner":1}
df["Ownership"]=df["Ownership"].map(o)
df["Fuel"]=df["Fuel"].map({"Petrol":0,"Diesel":0.5,"CNG":1})

input_col=["Year of Manufacture","Brand","Name","Transmission","Distance Travelled(km)","Ownership","Fuel"]
num_col=["Year of Manufacture","Distance Travelled(km)"]
cat_col=["Brand","Name"]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(df[num_col])
df[num_col]=scaler.transform(df[num_col])

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(df[cat_col])
encoded_cols = list(encoder.get_feature_names(cat_col))
df[encoded_cols] = encoder.transform(df[cat_col])

data=df[["Year of Manufacture","Transmission","Distance Travelled(km)","Ownership","Fuel"]+encoded_cols]
data

In [ ]:
target=df[['Price(Rs)']]
target

### Step 4: Applying Gradient Boosting Method

Libraries Used: Scikit-Learn, XGBoost

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
X_train, X_val, train_targets, val_targets = train_test_split(data, target, test_size=0.2)

In [ ]:
def Accuracy_RMSE(target,preds):
    target["Prediction"]=preds
    target["Accuracy"]=100-(abs(((target["Prediction"]-target["Price(Rs)"])/target["Price(Rs)"])*100))
    target["Squared Difference"]=(abs(target["Prediction"]-target["Price(Rs)"]))**2
    a,b=target["Accuracy"].mean(),(target["Squared Difference"].mean())**(0.5)
    target.drop(["Prediction","Accuracy","Squared Difference"],axis=1)
    return a,b

def test_maxdepth(x):
    val_rmse=[]
    val_accuracy=[]
    train_rmse=[]
    train_accuracy=[]
    for i in range(4,31,2):
        model = XGBRegressor(n_jobs=-1, random_state=42,n_estimators=x, max_depth=i)
        model.fit(X_train, train_targets[["Price(Rs)"]])
        a,b=Accuracy_RMSE(train_targets[["Price(Rs)"]],model.predict(X_train))
        c,d=Accuracy_RMSE(val_targets[["Price(Rs)"]],model.predict(X_val))
        val_rmse.append(d)
        val_accuracy.append(c)
        train_rmse.append(b)
        train_accuracy.append(a)
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle('Comparing RMSE and Accuracy')
    sns.lineplot(ax=axes[0], x=list(range(4,31,2)), y=val_rmse,label="Validation").set_title("RMSE")
    sns.lineplot(ax=axes[1], x=list(range(4,31,2)), y=val_accuracy,label="Validation").set_title("Accuracy")
    sns.lineplot(ax=axes[0], x=list(range(4,31,2)), y=train_rmse,label="Train").set_title("RMSE")
    sns.lineplot(ax=axes[1], x=list(range(4,31,2)), y=train_accuracy,label="Train").set_title("Accuracy")

In [ ]:
for x in range(5,26,5):
    print("n_estimators=",x)
    test_maxdepth(x)

In [ ]:
model = XGBRegressor(n_jobs=-1, random_state=42,n_estimators=20, max_depth=20)
model.fit(X_train, train_targets[["Price(Rs)"]])
preds = model.predict(X_val)
Accuracy_RMSE(val_targets[["Price(Rs)"]],preds)

### Step 5: Predicting Car Price

In [ ]:
import warnings
warnings.filterwarnings('ignore')

print("Input Details")
Name=input("Car Name:")
Brand=input("Car Brand:")
Year=int(input("Year of Manufacture:"))
Distance=int(input("Distance Travelled(km):"))
Ownership=input("Ownership:")
Transmission=input("Manual/Automatic:")
Fuel=input("Fuel Type:")

data = {
    'Year of Manufacture': [Year],
    'Brand': [Brand],
    'Name': [Brand+" "+Name],
    'Transmission': [Transmission],
    'Distance Travelled(km)': [Distance],
    'Ownership': [Ownership],
    'Fuel': [Fuel]
}

df = pd.DataFrame(data)

df["Transmission"]=df["Transmission"].map({"Manual":0,"Automatic":1})
o={"1st Owner":0,"2nd Owner":0.5,"3rd Owner":1}
df["Ownership"]=df["Ownership"].map(o)
df["Fuel"]=df["Fuel"].map({"Petrol":0,"Diesel":0.5,"CNG":1})

input_col=["Year of Manufacture","Brand","Name","Transmission","Distance Travelled(km)","Ownership","Fuel"]
num_col=["Year of Manufacture","Distance Travelled(km)"]
cat_col=["Brand","Name"]

df[num_col]=scaler.transform(df[num_col])

df[encoded_cols] = encoder.transform(df[cat_col])

data=df[["Year of Manufacture","Transmission","Distance Travelled(km)","Ownership","Fuel"]+encoded_cols]

price=list(model.predict(data))[0]
print("\n\n\nPrice of Car: Rs",price)